In [39]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


#https://colab.research.google.com/drive/1N3LvAO0AXV4kBPbTMX866OwJM9YS6Ji2?usp=sharing#scrollTo=fl5W1gg5Jhzz

df_players = pd.read_csv("./../data_scrapped/atp_players.csv")
df_matchs = pd.read_csv("./../data_formatted/training_dgl_dataset.csv")

In [41]:
#On construit le graph

# lecture des données attendues
# tensor1[1] -> tensor2[1] 
# le player index en tensor1 a joué contre le player au meme index en tensor 2
# y stocke le résultat du match


tensor1=[]
tensor2=[]

for index,row in df_players.iterrows():
    winmatchs = df_matchs[df_matchs.player1_id == row.player_id]
    #print(len(winmatchs))
    if len(winmatchs) > 0:
        #print(row.player_id)
        for index2, row2 in winmatchs.iterrows():
                tensor1.append(index)
                tensor2.append(df_players.loc[df_players.player_id == row2.player2_id].index[0])

In [42]:
import numpy
import random

labels = numpy.ones(len(tensor1))

indexes = random.sample(range(0, len(tensor1)-1), ((len(tensor1)-1)//2))

for index in indexes:
    temp = tensor1[index]
    tensor1[index] = tensor2[index]
    tensor2[index] = temp
    labels[index] = 0

In [55]:
import torch_geometric.transforms as T
from torch_geometric.data import Data

# Create the heterogeneous graph data object:
#data = Data()

# Add the user nodes:
x = torch.tensor(list(df_players[["birth_year","weight_kg","height_cm"]].values),dtype=torch.float)  # [num_users, num_features_users]
x = torch.nan_to_num(x, nan=0.0)
#x = torch.masked_select(x, ~torch.isnan(x))
#x = torch.ones(df_players.shape[0])

edge_index = torch.stack([torch.tensor(tensor1), torch.tensor(tensor2)], dim=0)
#edge_attr = torch.Tensor(list(df_matchs[["player1_atprank","player1_oddsB365","player2_atprank","player2_oddsB365"]].values))
#y = torch.Tensor(list(df_matchs[["winner_player1"]].values))

labels = torch.tensor(labels, dtype=torch.long)

data = Data(x=x, edge_index=edge_index,edge_label=labels) 
#data = T.ToUndirected()(data)
# Add the movie nodes:
#data['movie'].x = movie_features  # [num_movies, num_features_movies]

data

C:\Users\chbou\AppData\Local\Temp\ipykernel_15768\3264489422.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)


Data(x=[3446, 3], edge_index=[2, 51981], edge_label=[51981])

In [12]:
labels

tensor([1., 0., 1.,  ..., 0., 1., 1.])

In [122]:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.05,
    num_test=0.1,
    is_undirected=False,
    add_negative_train_samples=False,
    neg_sampling_ratio=0
)(data)
train_data, val_data, test_data

(Data(x=[3446, 3], edge_index=[2, 44184], edge_label=[44184], edge_label_index=[2, 44184]),
 Data(x=[3446, 3], edge_index=[2, 44184], edge_label=[2599], edge_label_index=[2, 2599]),
 Data(x=[3446, 3], edge_index=[2, 46783], edge_label=[5198], edge_label_index=[2, 5198]))

In [27]:
labels

tensor([1, 0, 1,  ..., 0, 1, 1])

In [104]:
print(train_data.edge_label.unique())
print(test_data.edge_label.unique())

tensor([0, 1])
tensor([0, 1])


In [32]:
train_data.edge_index

tensor([[2083,  242, 2642,  ..., 2342, 2061, 1530],
        [2624, 3252, 1093,  ..., 1608, 2321,  979]])

In [38]:
train_data.edge_index

tensor([[2083,  242, 2642,  ..., 2342, 2061, 1530],
        [2624, 3252, 1093,  ..., 1608, 2321,  979]])

In [123]:
from torch_geometric.nn import SAGEConv,MLP

if torch.cuda.is_available():
    device = torch.device('cuda')
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        self.mlp = MLP(in_channels=6, hidden_channels=hidden_channels, out_channels=1, num_layers=3)
        #self.fc = nn.MLP(6, 1) # Adjust output size for edge direction prediction

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return self.mlp(torch.cat((x[edge_label_index[0]], x[edge_label_index[1]]), dim=1))
    #(z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    #def decode_all(self, z):
    #    prob_adj = z @ z.t()
    #    return (prob_adj > 0).nonzero(as_tuple=False).t()


model = Net(3, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [66]:
x[train_data.edge_label_index[0]]

tensor([[1990.,   70.,  180.],
        [1986.,   79.,  183.],
        [1982.,   73.,  183.],
        ...,
        [1978.,   80.,  180.],
        [1987.,   77.,  188.],
        [1981.,   80.,  183.]])

In [143]:

def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    out = model.decode(z, train_data.edge_label_index).view(-1).sigmoid()
    loss = criterion(out.squeeze(), train_data.edge_label.float())
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    print(out)
    return roc_auc_score(data.edge_label.float(), out.squeeze())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')


tensor([1.3668e-16, 6.9912e-12, 1.9721e-17,  ..., 3.3901e-14, 2.3265e-13,
        2.0311e-16])
tensor([1.6091e-09, 3.1186e-15, 2.7541e-14,  ..., 3.6895e-12, 1.3356e-13,
        6.0586e-08])
Epoch: 001, Loss: 0.6787, Val: 0.5111, Test: 0.5193
tensor([1.3641e-16, 4.8731e-12, 3.6002e-17,  ..., 2.4668e-14, 1.5622e-13,
        2.4725e-16])
tensor([1.5509e-09, 1.6812e-14, 1.7702e-14,  ..., 2.6637e-12, 9.3676e-14,
        5.0381e-08])
Epoch: 002, Loss: 0.6786, Val: 0.5111, Test: 0.5204
tensor([2.2358e-16, 4.1640e-12, 6.6721e-17,  ..., 2.9901e-14, 1.2319e-13,
        1.0745e-15])
tensor([2.0859e-09, 7.2372e-14, 1.3140e-14,  ..., 2.2000e-12, 7.6545e-14,
        4.8914e-08])
Epoch: 003, Loss: 0.6786, Val: 0.5115, Test: 0.5217
tensor([4.6593e-16, 4.6264e-12, 1.2254e-16,  ..., 4.5240e-14, 1.5426e-13,
        3.6067e-15])
tensor([3.4480e-09, 2.3045e-13, 1.1573e-14,  ..., 2.3456e-12, 8.5797e-14,
        5.7018e-08])
Epoch: 004, Loss: 0.6785, Val: 0.5117, Test: 0.5226
tensor([9.4526e-16, 6.6566e-12, 

In [140]:
# Specify the edge you want to predict (e.g., edge from node 0 to node 1)
node_0 = 3
node_1 = 664

print(df_players.iloc[node_0])
#print(df_players.iloc[node_0]["first_name","last_name"])

# Predict the direction of the edge
with torch.no_grad():
    model.eval()
    edge_label_index =  torch.tensor([[node_0, node_1], [node_1, node_0]], dtype=torch.long).t()
    print(edge_label_index)
    z = model.encode(data.x, data.edge_index)
    prediction = model.decode(z, edge_label_index).view(-1).sigmoid()
    print(prediction)

player_id                l836
first_name               Nick
last_name             Lindahl
country_code              AUS
birthdate          1988.07.31
birth_year             1988.0
birth_month               7.0
birth_day                31.0
turned_pro             2006.0
weight_kg                77.0
weight_lbs              170.0
height_cm               183.0
height_in                72.0
birthplace      Malmo, Sweden
Name: 3, dtype: object
tensor([[  3, 664],
        [664,   3]])
tensor([0.6593, 0.7405])
